In [ ]:
import nltk
from nltk import corpus
from nltk.collocations import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import wordnet
from nltk import punkt
from nltk.util import ngrams
from nltk import FreqDist
#if these last 2 don't work might need to do nltk.download(package)
import re
from bs4 import BeautifulSoup

from string import punctuation
from collections import Counter
import numpy as np

In [ ]:
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

import pandas as pd
from datetime import datetime

#Importing sklearn for bigram finder
from sklearn.feature_extraction.text import CountVectorizer

## Import and Clean Connect Report

In [ ]:
report=pd.read_csv('connect_report.csv')

report['Created Date']=pd.to_datetime(report['Created Date'])
report=report[pd.notnull(report['Created Date'])]
report=report[pd.notnull(report['Body'])]

#Removing HTML tags and codes
report['Body'] = report['Body'].apply(lambda x:BeautifulSoup(x))
report['Body'] = report['Body'].apply(lambda x:x.get_text())

#Removing tags/mentions
report['Body'] = report['Body'].apply(lambda x:re.sub('{@[\w\d]*}', '',x))

## Report Generator

In [ ]:
final_report=pd.DataFrame

#Use this function to create the final report
# Directions:
# 1. Set Group to a list of group names, a single name of a group, or All to use all groups
# 2. Set Type to a specific Feed Item Type or All
# 3. Set Start Date or leave blank(defaults to: )
# 4. Set End Date or leave blank(defaults to: )

def report_generator(Group,Type,Date_start=pd.to_datetime("2017-01-01"),Date_finish=pd.to_datetime('today')):
    if type(Group) == list:
        final_report=report.loc[report['Name'].isin(Group)] 
    elif (Group=='All'):
        final_report=report
    else:
        final_report=report.loc[report['Name']==Group]
                                
    if (Type=='All'):
        final_report=final_report
    else:
        final_report=final_report.loc[final_report['Feed Item Type']==Type]
    
    if(Date_start=="2017-01-01" and Date_finish== pd.to_datetime('today')):
        final_report=final_report
    else:
        Date_start=pd.to_datetime(Date_start, utc=False)
        Date_finish=pd.to_datetime(Date_finish, utc=False)
        #alt approach-- not working but not sure why-- it worked it worked it worked! had to change the data type to all be annoying ass datatime64... down the line that might be a pain but for now it's dope
        final_report=final_report[(final_report['Created Date'] > Date_start) & (final_report['Created Date']< Date_finish)]   
    return(final_report)

final_report=report_generator(['CPC+ All','NLT Internal Users'],'All',"2017-02-01","2017-03-01")

final_report.head(5)

## Tokenizer Function

In [ ]:
##Stop word and lemmatizer creation (PLEASE add to stop where you see fit)
stop = stopwords.words('english')
stop.append("cpc+")
stop.append("hi")
stop.append("hello")
stop.append("also")
stop.append("anyone")

wnl = WordNetLemmatizer() 

In [ ]:
#this function tokenizes text, removes stop word and punctuation, and lemmatizes the tokens
def tokenizer(text):
 
    tokens_ = [word_tokenize(sent) for sent in sent_tokenize(text.lower())]
    
    tokens = []
    
    for token_by_sent in tokens_:
        tokens += token_by_sent
 
    tokens = list(filter(lambda t: t not in stop, tokens))
    tokens = list(filter(lambda t: t not in punctuation, tokens))
    tokens = list(filter(lambda t: t not in [u"'s", u"n't", u"...", u"''", u'``', u'\u2014', u'\u2026', u'\u2013'], tokens))
     
    filtered_tokens = []
    for token in tokens:
        token = wnl.lemmatize(token)
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
 
    filtered_tokens = list(map(lambda token: token.lower(), filtered_tokens))
 
    return filtered_tokens

In [ ]:
#tokenizes final report
final_report['tokens'] = final_report['Body'].map(tokenizer)
final_report['tokens'][0:10]

In [ ]:
##Bi-Grams using Sci-Kit Learn
##Finding bigrams and frequencies across all data

unlisted_report=final_report['tokens'].apply(lambda x: ', '.join(x))

word_vectorizer_2=CountVectorizer(ngram_range=(2,2), analyzer='word')
word_vectorizer_fit_2=word_vectorizer_2.fit_transform(unlisted_report)

frequencies=sum(word_vectorizer_fit_2).toarray()[0]

bigrams_counter=pd.DataFrame(frequencies,index=word_vectorizer_2.get_feature_names(),columns=['frequency'])
f = bigrams_counter[bigrams_counter['frequency']>2]

#The tokenizer cuts it down significantly to 639


##how can we integrate bigrams into the keywords functions-- we have identified the most common ones, how do we match them up with the original body comments?
print(word_vectorizer_fit_2.shape)
print(f[0:10])
#so they produce the same number of rows... but what's the 19354 columns?

In [ ]:
#extends tokens list with bi-grams for each row
#not lemmatized yet
for i,t in final_report['Body'].iteritems():
    t_list = t.lower().split()
    t_list = list(filter(lambda t: t not in stop, t_list))
    t_list = list(filter(lambda t: t not in punctuation, t_list))
    text=" ".join(t_list)
    
    ng = [ngrams(word_tokenize(sent),2) for sent in sent_tokenize(text.lower())]
    n_gram = []
    for n in ng:
            n_gram += n
    two_grams = []        
    for n in n_gram:
            new = ' '.join(n)
            two_grams.append(new)
    final_report['tokens'][i].extend(two_grams)

In [ ]:
final_report['tokens'][0:10]

## Keyword Retrieval

In [ ]:
#this function returns n keywords from specfied token list
def get_keywords(tokens, num):
    return Counter(tokens).most_common(num)

In [ ]:
#iterates through all text bodies and prints the raw text and the top 5 most common words
for i,t in final_report['tokens'].iteritems():
    print(final_report['Body'][i])
    print( get_keywords(t,10),"\n")

In [ ]:
#this functions prints the most common words by Feed Item Type
def keywords_bytype(feed_type):
    tokens = final_report[final_report['Feed Item Type'] == feed_type]['tokens']
    alltokens = []
    for token_list in tokens:
        alltokens += token_list
    counter = Counter(alltokens)
    return counter.most_common(10)

In [ ]:
#iterates through all text bodies abd prints most common words by Feed Item Type
for t in set(final_report['Feed Item Type']):
    print('category :', t)
    print('top 10 keywords:', keywords_bytype(t))
    print('---')

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# min_df is minimum number of documents that contain a term t
# max_features is maximum number of unique tokens (across documents) that we'd consider
# TfidfVectorizer preprocesses the descriptions using the tokenizer we defined above

In [ ]:
#create a vectorizer object for our data with min docs. = 2, max feats. = 10K, and single and paired words
vectorizer = TfidfVectorizer(min_df=2, max_features=10000, tokenizer=tokenizer, ngram_range=(1, 2))
vz = vectorizer.fit_transform(list(final_report['Body']))

In [ ]:
#create a dictionary mapping the tokens to their tfidf values
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
tfidf = pd.DataFrame(columns=['tfidf']).from_dict(dict(tfidf), orient='index')
tfidf.columns = ['tfidf']

In [ ]:
#low tfidf scores
tfidf.sort_values(by=['tfidf'], ascending=True).head(15)

In [ ]:
#high tfidf scores
tfidf.sort_values(by=['tfidf'], ascending=False).head(15)

## Correlated Words

In [ ]:
##Constructing a collocation finder-- this is getting closer, but a pretty shitty tokenizing job


#tokens_2=tokens_2.append(final_report['Body'].apply(word_tokenize))

finder=BigramCollocationFinder.from_documents(final_report['Body'].apply(word_tokenize))
bigram_measures=nltk.collocations.BigramAssocMeasures()
score_collocation=finder.nbest(bigram_measures.pmi,15)

print(score_collocation)


In [ ]:
#Using tokens

finder_2=BigramCollocationFinder.from_documents(final_report['tokens'])
score_collocation_2=finder_2.nbest(bigram_measures.pmi,5)

print(score_collocation_2)

In [ ]:
scored = finder_2.score_ngrams(bigram_measures.raw_freq)
sorted_scored=sorted(bigram for bigram, score in scored)
#So the top results are kind of crappy, is there a way to drill down or score in a way that makes more sense?
print(sorted_scored)

In [ ]:
#unlisted_report=final_report['tokens'].apply(lambda x: ', '.join(x))
type(sorted_scored)
##why isn't this working?
scored.index(('risk'))

In [ ]:
scored.index(('single','package'))

#have to do the whole thing which is super annoying

#so what if we do two words and see if they are colocated and then print the score.... or could loop both things through a keyword list